# Effect size extrapolation

In the [last notebook], we calculated emperical and distribution-based power for five types of statistical tests:
* One Sample T test
* Independent Sample t test 
* One way ANOVA, 3 groups
* One way ANOVA, 8 groups
* Linear correlation

We will now evaluate 

In [ ]:
import copy
import os
import pickle
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sn
import scipy
# import statsmodels.api as sms
import statsmodels.formula.api as smf

import effects as eff
import plot as plot

% matplotlib inline
sn.set_style('ticks')

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
num_rounds = 100
alpha = 0.05
counts = np.arange(5, 100, 10)
colormap = 'Spectral'
overwrite = True

In [ ]:
sim_location = './simulations'
if not os.path.exists(sim_location):
    raise ValueError('The power simulations do not exist.'
                     'Please go back to notebooks 2 and 3 and'
                     'calculate power.'
                     )

In [ ]:
colors = sn.color_palette(colormap, n_colors=len(counts))
check_color = {count: colors[i] for i, count in enumerate(counts)}

In [ ]:
tests = ['permanova', 'mantel']
# tests = ['ttest_1']

In [ ]:
extrapolated = ['f_power', 't_power', 'z_power']

In [ ]:
distributions = {'permanova': {'clean_name': 'PERMANOVA',
                               'num_groups': 2,
                               'input_dir': './simulations/power/permanova/',
                               'return_fp': './simulations/extrapolation/permanova.txt'
                               },
                 'mantel': {'clean_name': 'Mantel',
                            'num_groups': 2,
                            'input_dir': './simulations/power/mantel/',
                            'return_fp': './simulations/extrapolation/mantel.txt'
                            },
                 }

In [ ]:
return_dir = './simulations/extrapolation/'
if not os.path.exists(return_dir):
    os.makedirs(return_dir)

We'll start by calculating the emperical and extrapolated effect sizes for the parametric tests.

In [ ]:
for test_name in tests:
    power_dir = distributions[test_name]['input_dir']
    num_groups = distributions[test_name]['num_groups']
    return_fp = distributions[test_name]['return_fp']
    
    if not os.path.exists(power_dir):
        raise ValueError('%s does not exist' % power_dir)
        
    summaries = []
     # Loops through the rounds
    for i in range(5):
        

In [ ]:
for test_name in tests:
    power_dir = distributions[test_name]['input_dir']
    num_groups = distributions[test_name]['num_groups']
    return_fp = distributions[test_name]['return_fp']
    
    if not os.path.exists(power_dir):
        raise ValueError('%s does not exist' % power_dir)
        
    summaries = []
    
    # Loops through the rounds
    for i in range(num_rounds):
        # Loads through the power simulation for the round
        power_fp = os.path.join(power_dir, 'simulation_%i.p' % i)
        
        with open(power_fp, 'rb') as f_:
            sim = pickle.load(f_)
        
        # Pulls the previously calculated power values
        counts = sim['counts']
        emperical = sim['emperical_power']
        empr_shape = emperical.shape
        z_effect = eff.z_effect(counts, emperical, alpha=0.05)
        t_effect = eff.t_effect(counts, emperical, alpha=0.05)
        f_effect = eff.f_effect(counts, emperical, groups=3, alpha=0.05)
        num_obs = (empr_shape[0] * empr_shape[1])
        run_summary = pd.DataFrame({
                    'counts': np.hstack([counts] * empr_shape[0]),
                    'emperical_power': np.hstack(emperical),
                    'sim_pos': np.hstack([np.arange(empr_shape[1]) + (i + 1) * 10 
                                          for i in range(empr_shape[0])]),
                    'z_effect': np.hstack(z_effect),
                    't_effect': np.hstack(t_effect),
                    'f_effect': np.hstack(f_effect),
                    'z_power': np.hstack([eff.z_power(counts, np.nanmean(z_effect))]
                                         * empr_shape[0]),
                    't_power': np.hstack([eff.t_power(counts, np.nanmean(t_effect))]
                                         * empr_shape[0]),
                    'f_power': np.hstack([eff.f_power(counts, np.nanmean(f_effect), groups=num_groups)]
                                         * empr_shape[0]),
                    })
        run_summary['color'] = run_summary['counts'].apply(lambda x: check_color[x])
        run_summary['test'] = test_name
        run_summary['clean_name'] = distributions[test_name]['clean_name']
        run_summary['simulation'] = i
        run_summary['fit_f-mean'] = np.nanmean(f_effect)
        run_summary['fit_f-std'] = np.nanstd(f_effect)
        run_summary['fit_f-count'] = np.sum(np.isnan(f_effect) == False) / num_obs
        run_summary['fit_t-mean'] = np.nanmean(t_effect)
        run_summary['fit_t-std'] = np.nanstd(t_effect)
        run_summary['fit_t-count'] = np.sum(np.isnan(t_effect) == False) / num_obs
        run_summary['fit_z-mean'] = np.nanmean(z_effect)
        run_summary['fit_z-std'] = np.nanstd(z_effect)
        run_summary['fit_z-count'] = np.sum(np.isnan(z_effect) == False) / num_obs
        run_summary['index'] = (run_summary['test'] + '.' +  
                                run_summary['simulation'].apply(lambda x: '%i' % x) + '.' +
                                run_summary['sim_pos'].apply(lambda x: '%i' % x))
        run_summary.set_index('index', inplace=True)
        summaries.append(pd.DataFrame(run_summary))
    summaries = pd.concat(summaries)
    summaries.to_csv(return_fp, sep='\t')
    distributions[test_name]['summary'] = summaries

For most of the effect sizes and fits, we find the behavior of the curve 

Let's also compare the behavior of the emperical power and the fit power curves. 

In [ ]:
# Sets up the figure and axes
er_fig, er_axes = plt.subplots(3, 4)
er_fig.set_size_inches(8, 6)
sn.despine()

for ax in er_axes[:, 2:].flatten():
    ax.set_visible(False)

for idc, test_name in enumerate(tests):
    summary = distributions[test_name]['summary']
    for metric, ax_reg in zip(*[extrapolated, er_axes.T[idc]]):
        plot.gradient_regression(ax=ax_reg, 
                            x='emperical_power', 
                            y=metric, 
                            gradient='color', 
                            alpha=0.25,
                            data=summary
                            )
        plot.format_regression_axis(ax_reg)
        if metric == 'z_power':
            ax_reg.set_xticklabels(ax_reg.get_xticks())
        if metric == 'f_power':
            ax_reg.set_title(distributions[test_name]['clean_name'])
        if test_name == tests[0]:
            ax_reg.set_yticklabels(ax_reg.get_yticks())
            ax_reg.set_ylabel(metric.replace('_', ' ').capitalize())
        
er_axes[-1][2].set_xlabel('Emperical Power')

It looks like both permutative distributions lead to a curves correlation. Some of that behavior may be shaped by the effects at lower and upper power, where the effect size approximation is difficult. So, we'll try recalculating the effect sizes and power exlcuding power less than 0.1, or power greater than 0.95 in the effect size calculations.

In [ ]:
def calc_series_f(x):
    pwr = eff.f_power(np.array([x['counts']]), np.array([x['fit_f-mean']]))
    return pwr[0]
def calc_series_t(x):
    pwr = eff.t_power(np.array([x['counts']]), np.array([x['fit_t-mean']]))
    return pwr[0]
def calc_series_z(x):
    pwr = eff.z_power(np.array([x['counts']]), np.array([x['fit_t-mean']]))
    return pwr[0]

def calc_modified_effect(drop_index, summary):
    """..."""
    copy_cols = ['emperical_power', 'counts', 'color', 
                 'clean_name', 'simulation', 'sim_pos', 'test',
                 'f_effect', 't_effect', 'z_effect']
    summary_mod = copy.deepcopy(summary[copy_cols])
    summary_mod.loc[drop_index,  ['f_effect', 't_effect', 'z_effect']] = np.nan
    mean_lookup = summary_mod.groupby('simulation').mean()[['f_effect', 't_effect', 'z_effect']].to_dict()
    std_lookup = summary_mod.groupby('simulation').std()[['f_effect', 't_effect', 'z_effect']].to_dict()
    
    summary_mod['fit_f-mean'] =  summary_mod['simulation'].replace(mean_lookup['f_effect'])
    summary_mod['fit_t-mean'] =  summary_mod['simulation'].replace(mean_lookup['t_effect'])
    summary_mod['fit_z-mean'] =  summary_mod['simulation'].replace(mean_lookup['z_effect'])
    
    summary_mod['fit_f_std'] = summary_mod['simulation'].replace(std_lookup['f_effect'])
    summary_mod['fit_t_std'] = summary_mod['simulation'].replace(std_lookup['t_effect'])
    summary_mod['fit_z_std'] = summary_mod['simulation'].replace(std_lookup['z_effect'])
    
    summary_mod['f_power'] = summary_mod.apply(calc_series_f, axis=1)
    summary_mod['t_power'] = summary_mod.apply(calc_series_t, axis=1)
    summary_mod['z_power'] = summary_mod.apply(calc_series_z, axis=1)
    
    return summary_mod

In [ ]:
for idc, test_name in enumerate(['permanova']):
    original = distributions[test_name]['summary']
    drop_index = ((original['emperical_power'] < 0.1) | 
                  (original['emperical_power'] > 0.9)
                  )
    summary = calc_modified_effect(drop_index, original)
    for metric, ax_reg in zip(*[extrapolated, er_axes.T[idc + 2]]):
        ax_reg.set_visible(True)
        plot.gradient_regression(ax=ax_reg, 
                            x='emperical_power', 
                            y=metric, 
                            gradient='color', 
                            alpha=0.25,
                            data=summary
                            )
        plot.format_regression_axis(ax_reg)
        if metric == 'z_power':
            ax_reg.set_xticklabels(ax_reg.get_xticks())
        if metric == 'f_power':
            ax_reg.set_title('%s\n(limited)' % distributions[test_name]['clean_name'])

er_fig